In [62]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
import pandas as pd
import numpy as np

class UnexpectedAlertPresentException(Exception):
    pass

batch = '21'
branch = 'ME'
final_number = 11

df = pd.DataFrame()

driver_path = 'C:\dev\chromedriver.exe'

driver = webdriver.Chrome(driver_path)

url = 'https://results.vtu.ac.in/JFEcbcs23/index.php'

driver.get(url)

k=1

while k<=final_number:
    usn = '1AM'+batch+branch+f'{k}'.zfill(3)

    usn_bar = driver.find_element_by_name('lns')
    usn_bar.send_keys(usn)

    captcha_bar = driver.find_element_by_name('captchacode')
    captcha_bar.click()

    try:
        WebDriverWait(driver, timeout=100).until(lambda d: d.find_element_by_xpath('//*[@id="dataPrint"]/div[2]/div/div/div[2]/div[3]/div[3]/table/tbody/tr[2]/td/b[2]'))
    except:
        alert = WebDriverWait(driver, timeout=5).until(expected_conditions.alert_is_present())

        if alert.text == 'Invalid captcha code !!!':
            alert.accept()
            continue
        elif alert.text == 'University Seat Number is not available or Invalid..!':
            k+=1
            alert.accept()
            continue
    
    student_name = driver.find_element_by_xpath('//*[@id="dataPrint"]/div[2]/div/div/div[2]/div[1]/div/div/div[1]/div/table/tbody/tr[2]/td[2]').text.split(':')[1].strip()
    student_usn = driver.find_element_by_xpath('//*[@id="dataPrint"]/div[2]/div/div/div[2]/div[1]/div/div/div[1]/div/table/tbody/tr[1]/td[2]').text.split(':')[1].strip()

    assert student_usn == usn, 'entered usn and obtained usn don\'t match'

    marks_data = driver.find_element_by_xpath('//*[@id="dataPrint"]/div[2]/div/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div[2]')

    data_array = np.zeros(shape=(9,7), dtype=object)

    row_tags = marks_data.find_elements_by_css_selector('.divTableRow')

    for i in range(len(row_tags)):
        data_array[i] = [x.text for x in row_tags[i].find_elements_by_css_selector('.divTableCell')]

    data_array = data_array[:,:-1]

    if k == 1:
        header_names = data_array[0][2:]
        subject_names = data_array[:,1][1:]

        list_of_tuples = []

        for i in subject_names:
            for j in header_names:
                list_of_tuples.append((i, j))

    df[student_usn] = np.concatenate((np.array([student_name]), data_array[1:,2:].flatten()))

    driver.back()

    k+=1

driver.quit()

In [100]:
df2 = df.T.reset_index()
df2.columns = [('USN',''),('Student Name','')] + list_of_tuples
df2.columns = pd.MultiIndex.from_tuples(df2.columns, names=['',''])
df2.index+=1

df2.to_excel(f'{batch} {branch} results.xlsx')

In [101]:
df2

USN          Student Name  \
                                       
1   1AM21ME001  AMITH KUMAR N RATHOD   
2   1AM21ME002           BASAVARAJ T   
3   1AM21ME003           CHETHAN S N   
4   1AM21ME004             HEMANTH V   
5   1AM21ME006        MAHESH GOWDA S   
6   1AM21ME007       PRATHAP KUMAR R   
7   1AM21ME008            RAKESH M N   
8   1AM21ME009          S SAI CHARAN   
9   1AM21ME010                TEJA C   
10  1AM21ME011             VINAY C B   

   TRANSFORM CALCULUS, FOURIER SERIES AND NUMERICAL TECHNIQUES                 \
                                                Internal Marks External Marks   
1                                                  32                      19   
2                                                  22                       0   
3                                                  30                      23   
4                                                  22                      18   
5                                                  22                      10   
6                                                  38                      28   
7                                                  37                      21   
8                                                  22                       3   
9                                                  22                       9   
10                                                 31                       6   

                FUNDAMENTALS OF VIRTUAL REALITY                              \
   Total Result                  Internal Marks External Marks Total Result   
1     51      P                              48             30    78      P   
2     22      A                        NE (0+0)             NE     -      X   
3     53      P                              48             26    74      P   
4     40      P                              46             38    84      P   
5     32      F                              47             21    68      P   
6     66      P                              44             20    64      P   
7     58      P                              45             19    64      P   
8     25      F                              44             37    81      P   
9     31      F                              47             20    67      P   
10    37      F                              48             19    67      P   

    ... MACHINE DRAWING AND GD & T        SOCIAL CONNECT AND RESPONSIBILITY  \
    ...                      Total Result                    Internal Marks   
1   ...                         73      P                                44   
2   ...                          -      X                            NE (0)   
3   ...                         69      P                                40   
4   ...                         68      P                                42   
5   ...                         57      P                                35   
6   ...                         87      P                                45   
7   ...                         61      P                                42   
8   ...                         56      P                                40   
9   ...                         34      A                                36   
10  ...                         60      P                                36   

                               SAMSKRUTHIKA KANNADA                       \
   External Marks Total Result       Internal Marks External Marks Total   
1              49    93      P                   32             28    60   
2              NE     -      X               NE (0)             NE     -   
3              45    85      P                   29             22    51   
4              42    84      P                   25             20    45   
5              43    78      P                   26             18    44   
6              38    83      P                   31             21    52   
7              33    75      P                   37